In [ ]:
# ZANIM ZACZNIEMY MUSIMY ZAINSTALOWAĆ PYDANTIC, BYĆ MOŻE BĘDZIEMY MUSIELI ZROBIĆ RESTART KERNELA
!pip install --quiet instructor pydantic openai

In [ ]:
import json
from pathlib import Path
from datetime import date
import base64
from getpass import getpass

from IPython.display import Image
import instructor
from pydantic import BaseModel
from openai import OpenAI
import pandas as pd

In [ ]:
openai_key = getpass("Wprowadź swój klucz OpenAI: ")

In [ ]:
openai_client = OpenAI(api_key=openai_key)

# Pracujemy z plikami - PDF / Obrazek / Zrzut ekranu

## ZANIM ZACZNIEMY

* Pod video załączone są linki do kilku rachunków za gaz w formacie `.png`.
* Pobierz te pliki.
* Następnie utwórz folder `dane_gaz`
* A w nim foldery `raw` i `processed`
* W folderze `raw` umieść pobrane rachunki

In [ ]:
RAW_DATA_PATH = Path("dane_gaz") / "raw"

PROCESSED_DATA_PATH = Path("dane_gaz") / "processed"

In [ ]:
for image_path in RAW_DATA_PATH.glob("*.png"):
    print(image_path)

## Przykładowy rachunek

In [ ]:
Image(RAW_DATA_PATH / "gaz_2023_12.png")

## Przygotowanie obrazków do przesłania do OpenAI

In [ ]:
image_path = RAW_DATA_PATH / "gaz_2023_12.png"

with open(image_path, "rb") as f:
    image_data = base64.b64encode(f.read()).decode('utf-8')

image_data[:100]

In [ ]:
def prepare_image_for_open_ai(image_path):
    with open(image_path, "rb") as f:
        image_data = base64.b64encode(f.read()).decode('utf-8')

    return f"data:image/png;base64,{image_data}"

## Wyciągamy dane przy pomocy OpenAI

In [ ]:
image_path = RAW_DATA_PATH / "gaz_2023_12.png"

response = openai_client.chat.completions.create(
    # model="gpt-4o",
    model="gpt-4o-mini",
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": """
wyciągnij wszystkie informacje zawarte na fakturze.
Dane przedstaw w formacie JSON.
Oczekuję następujących informacji:
{
"razem_sprzedaż_okres_rozliczeniowy_data_od": ...,
"razem_sprzedaż_okres_rozliczeniowy_data_do": ...,
"zużycie_m3": ...,
"zużycie_kWh": ...,
"do_zapłaty": ...,
"termin_płatności": ...,
}
tylko dane jako JSON, bez żadnych komentarzy
"""
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": prepare_image_for_open_ai(image_path),
                        "detail": "high"
                    },
                },
            ],
        }
    ],
)

# {
# "razem_sprzedaż_okres_rozliczeniowy_data_od": "01.12.2023",
# "razem_sprzedaż_okres_rozliczeniowy_data_do": "31.12.2023",
# "zużycie_m3": 130,
# "zużycie_kWh": 1505,
# "do_zapłaty": "463,55 zł",
# "termin_płatności": "18.12.2023"
# }
print(response.choices[0].message.content)

In [ ]:
for image_path in RAW_DATA_PATH.glob("*.png"):
    print(f"Processing {image_path}")

    response = openai_client.chat.completions.create(
        model="gpt-4o",
        # model="gpt-4o-mini",
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": """
wyciągnij wszystkie informacje zawarte na fakturze.
Dane przedstaw w formacie JSON.
Oczekuję następujących informacji:
{
    "razem_sprzedaż_okres_rozliczeniowy_data_od": ...,
    "razem_sprzedaż_okres_rozliczeniowy_data_do": ...,
    "zużycie_m3": ...,
    "zużycie_kWh": ...,
    "do_zapłaty": ...,
    "termin_płatności": ...,
}
tylko dane jako JSON, bez żadnych komentarzy
"""
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": prepare_image_for_open_ai(image_path),
                            "detail": "high"
                        },
                    },
                ],
            }
        ],
    )

    result = response.choices[0].message.content.replace("```json", "").replace("```", "").strip()
    with open(PROCESSED_DATA_PATH / f"{image_path.stem}__simple.json", "w") as f:
        f.write(result)

## Wyciągamy dane z pliku PDF przy pomocy OpenAI + instructor

In [ ]:
class GasBillInfo(BaseModel):
    okres_rozliczeniowy_od: date
    okres_rozliczeniowy_do: date
    zużycie_m3: float
    zużycie_kWh: float
    do_zapłaty: float
    termin_płatności: date

instructor_openai_client = instructor.from_openai(OpenAI(api_key=openai_key))

In [ ]:
image_path = RAW_DATA_PATH / "gaz_2023_12.png"

gas_bill = instructor_openai_client.chat.completions.create(
    model="gpt-4o",
    response_model=GasBillInfo,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Pobierz szczegóły rachunku za gaz",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": prepare_image_for_open_ai(image_path),
                        "detail": "high"
                    },
                },
            ],
        },
    ],
)

gas_bill

In [ ]:
for image_path in RAW_DATA_PATH.glob("*.png"):
    print(f"Processing {image_path}")

    gas_bill = instructor_openai_client.chat.completions.create(
        model="gpt-4o",
        response_model=GasBillInfo,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Pobierz szczegóły rachunku za gaz",
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": prepare_image_for_open_ai(image_path),
                            "detail": "high"
                        },
                    },
                ],
            },
        ],
    )
    with open(PROCESSED_DATA_PATH / f"{image_path.stem}.json", "w") as f:
        f.write(gas_bill.model_dump_json())


In [ ]:
# wczytajmy wszystkie dane
data = []
for json_path in PROCESSED_DATA_PATH.glob("*.json"):
    if "simple" in json_path.name:
        continue 
        
    with open(json_path) as f:
        data.append(json.loads(f.read()))

df = pd.DataFrame(data)
df

In [ ]:
df.sort_values("termin_płatności").plot(x="termin_płatności", y="do_zapłaty", kind="bar")